# Pull Beta Account Models

You can select training jobs in beta, which are meeted the business needs, to pull machine learning models into devops account. These models are saved in SageMaker model repository.

## Dev accounts

Login AWS console:
* aws login account: mlops-dev
* user: mlops-dev-developers

Open the cloud9 - my-mlops-works


# Pull model from beta

* mkdir pull-training-model
* aws --profile mlops-devops-developers codecommit create-repository --repository-name pull-training-model
* cd pull-training-model
* git init
* git remote add origin https://git-codecommit.us-west-2.amazonaws.com/v1/repos/pull-training-model
* git remote -v
* touch pull-training-model.py

The following is pull-training-model codes and commit the code into codecommit

### pull-training-model.py

In [ ]:
import boto3, sys
from urllib.parse import urlparse

# model name use the job name
beta_access_key = sys.argv[1]
beta_secrete_key = sys.argv[2]
devops_md_bucket = sys.argv[3]
job_name = sys.argv[4]

# beta sagemaker
ss_beta = boto3.Session(aws_access_key_id=beta_access_key, aws_secret_access_key=beta_secrete_key)
sm_beta = ss_beta.client('sagemaker')
s3_beta = ss_beta.client('s3')
resp = sm_beta.describe_training_job(TrainingJobName=job_name)

# devops sagemaker
sm = boto3.client('sagemaker')
s3 = boto3.client('s3')
account_devops = boto3.client('sts').get_caller_identity()['Account']

# copy model to devops s3
md_url_beta = resp['ModelArtifacts']['S3ModelArtifacts']
output = urlparse(md_url_beta)
beta_md_bucket = output.netloc
beta_md_key = output.path.lstrip('/')
source_response = s3_beta.get_object(Bucket=beta_md_bucket, Key=beta_md_key)
s3.upload_fileobj(source_response['Body'],devops_md_bucket,beta_md_key)

# create sagemaker model
image = resp['AlgorithmSpecification']['TrainingImage']
role = 'arn:aws:iam::{}:role/SageMakerExecutionRole'.format(account_devops)
modelDataUrl = 's3://{}/{}'.format(devops_md_bucket,beta_md_key)
job_arn = resp['TrainingJobArn']

create_model_response = sm.create_model(
    ModelName=job_name,
    ExecutionRoleArn=role,
    PrimaryContainer={
        'Image': image,
        'ModelDataUrl': modelDataUrl},
    Tags=[{"Key": "job_arn","Value": job_arn}])

print(create_model_response['ModelArn'])

# DevOps account

Login
* Account: mlops-devops
* IAM User: mlops-devops-admin
* Group: devops-admin

# Create S3 Bucket

* Bucket name: `devops-sagemaker-repo-us-west-2`

# Create SageMakerExecutionRole

* IAM
* RoleName: SageMakerExecutionRole
* Services: SageMaker
* Policy: SageMakerFullAccess

## Create CodeBuild Project
* Project Name: `devops-pull-model`
* Description: `Pull model into devops account.`
* source: pull-training-model
* image: amazonlinux2
* new service role: `codebuild-devops-pull-model-service-role`

## buildspec.yml

In [ ]:
version: 0.2

env:
  variables:
     devops_md_bucket: "devops model bucket"
     job_name: "traning job name in beta"
  secrets-manager:
     beta_access_key: beta/sagemaker:beta-sagemaker-access
     beta_secret_key: beta/sagemaker:beta-sagemaker-secret
phases:
  install:
    runtime-versions:
      python: 3.8
  build:
    commands:
      - python pull-training-model.py $beta_access_key $beta_secret_key $devops_md_bucket $job_name

## IAM

Find the role: `codebuild-devops-pull-model-service-role` and add inline policy `codebuild-pull-model`

In [ ]:
{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "VisualEditor0",
            "Effect": "Allow",
            "Action": [
                "s3:PutObject",
                "sagemaker:CreateModel",
                "iam:PassRole",
                "secretsmanager:GetSecretValue"
            ],
            "Resource": [
                "arn:aws:secretsmanager:*:*:secret:beta/sagemaker-*",
                "arn:aws:sagemaker:*:*:model/*",
                "arn:aws:iam::*:role/SageMakerExecutionRole",
                "arn:aws:s3:::devops-sagemaker-repo*/*"
            ]
        }
    ]
}

## Start build

## Evnironment variables override

* key: job_name,  value: <check previous codebuild project output>
* key: devops_md_bucket, value: `devops-sagemaker-repo-us-west-2`

![](./img/14.png)

In [ ]:
[Container] 2020/01/07 05:29:00 Entering phase INSTALL 
[Container] 2020/01/07 05:29:00 Phase complete: INSTALL State: SUCCEEDED 
[Container] 2020/01/07 05:29:00 Phase context status code:  Message:  
[Container] 2020/01/07 05:29:00 Entering phase PRE_BUILD 
[Container] 2020/01/07 05:29:00 Phase complete: PRE_BUILD State: SUCCEEDED 
[Container] 2020/01/07 05:29:00 Phase context status code:  Message:  
[Container] 2020/01/07 05:29:01 Entering phase BUILD 
[Container] 2020/01/07 05:29:01 Running command python pull-training-model.py $beta_access_key $beta_secret_key $mlops_account_id $job_name 
arn:aws:sagemaker:us-west-2:***:model/scikit-bring-your-own-2020-01-06-12-47-23 

# Congradulations

You finished the following redbox part in the architecture diagram:

![](./img/15.png)